# Effluent Logger App

In [7]:
# we want to display the amount of effluent in each paddock
# we need to provide a start and end date
# we then need to query each of those dates for their amount of effluent and which paddocks
# we then need to sum those values
# we then display the numbers

# imports
%matplotlib widget
import sqlite3
import pandas as pd
import plotly
import ipywidgets as widgets
import numpy as np
import cufflinks as cf
import dateutil.relativedelta
import matplotlib.pyplot as plt

from datetime import datetime,timedelta

from ipysheet import sheet, cell, row, column, cell_range, from_dataframe, to_dataframe, to_array
from IPython.display import clear_output

# Global Variables
# we have two databases, one for the raw inputs and one for the parsed data.
# EffluentInputLogs - raw inputs
# Fields
    # Date -  text, required - date spreading was logged
    # Start - text, required - start placement of pivot
    # Stop  - text, required - end placement of pivot
    # Speed - numeric, optional - operational speed of pivot as a percentage of max speed
    # Storage - text, optional  - unsure what this is about
    # Notes - text, optional - place to store notes from log
input_fields = ['Date', 'Start', 'Stop', 'Speed', 'Storage', 'Notes']

# EffluentLogs - parsed data log for result of effluent spreading on paddocks
# Fields
    # Date - text, required, foreign key - date spreading was logged
    # 2b, 2c, 3a, 3b, 4a, 4b, 5a, 5b, 6a, 6b, 7aL, 7aR, 7b - integer, requied - amount of effluent added to each paddock.
paddocks = ['2b', '2c', '3a', '3b', '4a', '4b', '5a', '5b', '6a', '6b', '7aL', '7aR', '7b']



## Database stuff

In [8]:
### Database class. Allows for simple connecting and disconnecting to a database ###
class Database():
    def __init__(self, database_path):
        self.path = database_path
        self.conn = None
        self.c = None
        self.connect()
        print('Database Connected')
        
    def updatePath(self, path):
        self.path = path
    
    def connect(self):
        self.conn = sqlite3.connect(self.path)
        self.c = self.conn.cursor()
        
    def commit(self):
        """commit changes to database"""
        self.conn.commit()
    
    def disconnect(self):
        self.c.close()
        self.conn.close()
        print('Connection Closed')
    
    def query(self, query):
        return self.c.execute(query)
    
    def queryTableDF(self, query):
        # Query a table in a database and return in a DataFrame format
        return pd.read_sql_query(query, self.conn)
        
    def queryValue(self, query, row=0, column=0):
        # Return a specific value from a row/column in a queried dataframe table.
        result_df = self.queryTableDF(query)
        return result_df.iat[row,column]

### Setup the database ###
#default_database_location = 'C:\Predictive Maintenance\Maintenance.db'
default_database_location = 'D:\Current Projects\Farm\Effluent Tracker\Effluent Database.db'
db = Database(default_database_location)

### Attempt a query from the database. If there is an error indicate to user that database is disconnected ###

connected_html = widgets.HTML('<p style="color:green"><b>Connected to %s</b></p>' % default_database_location)
disconnected_html = widgets.HTML('<p style="color:red"><b> Database not connected! Check database name/location: %s</b></p>' % default_database_location)

query = 'SELECT * FROM EffluentLogs'
try:
    result = db.query(query)
    connected = True
except:
    connected = False
    
if connected:
    display(connected_html)
else:
    display(disconnected_html)

Database Connected


HTML(value='<p style="color:green"><b>Connected to D:\\Current Projects\\Farm\\Effluent Tracker\\Effluent Data…

# Effluent Graph

In [9]:
def make_box_layout():
     return widgets.Layout(
        border='solid 1px black',
        margin='0px 10px 10px 0px',
        padding='5px 5px 5px 5px',
        width='95%'
     )
    

class EffluentTracker(widgets.VBox):
     
    def __init__(self):
        super().__init__()
        output = widgets.Output()
 
        self.end_date = datetime.today()
        self.start_date = self.end_date - dateutil.relativedelta.relativedelta(years=1)
        query=f'Select * FROM EffluentLogs WHERE Date BETWEEN "{self.start_date}" AND "{self.end_date}"'
        self.EffluentLog = db.queryTableDF(query)  
        
        # graph setup
        with output:
            plt.close('all')
            self.fig, self.ax = plt.subplots(constrained_layout=True, figsize=(9, 6))
            
            self.bar = self.ax.bar(paddocks, self.EffluentLog[paddocks].sum().tolist())
        
            self.fig.suptitle('Effluent Per Paddock', fontsize=16)
            self.fig.set_label(" ")
            self.fig.canvas.toolbar_position = 'bottom'
            self.ax.grid(False)
 
        # define widgets
        date_start = widgets.DatePicker(
            value=pd.to_datetime(self.start_date.strftime("%Y-%m-%d")),
            description='Start'
        )
        date_end = widgets.DatePicker(
            value=pd.to_datetime(self.end_date.strftime("%Y-%m-%d")), 
            description='End'
        )
 
        controls = widgets.HBox([
            date_start, 
            date_end
        ], 
            layout = widgets.Layout(width='auto',
                                    align_items='stretch')
        )
        controls.layout = make_box_layout()
         
        out_box = widgets.Box([output])
        output.layout = make_box_layout()
 
        # observe stuff
        date_start.observe(self.update_start_date, 'value')
        date_end.observe(self.update_end_date, 'value')   
 
        # add to children
        self.children = [controls, output]
        #self.children = widgets.VBox(self.children)
        
    def query_database(self, change):
        query=f'Select * FROM EffluentLogs WHERE Date BETWEEN "{self.start_date}" AND "{self.end_date}"'
        #print(query)
        self.EffluentLog = db.queryTableDF(query)  
        
    def update_graph(self, change):
        sums = self.EffluentLog[paddocks].sum().tolist()
        for bar,height in zip(self.bar, sums):
            bar.set_height(height)
        self.fig.canvas.draw()
        
    def update_start_date(self, change):
        """Draw line in plot"""
        self.start_date = change.new
        self.query_database(self)
        self.update_graph(self)
        
    def update_end_date(self, change):
        """Draw line in plot"""
        self.end_date = change.new
        self.query_database(self)
        self.update_graph(self)
         
EffluentTracker()

EffluentTracker(children=(HBox(children=(DatePicker(value=Timestamp('2021-01-03 00:00:00'), description='Start…

# Past Effluent Logs

In [10]:
# this section alows the logging of effluent records to the database
# should check what currently exists in effluent helper in spyder
# Ideally I think this should look give a look at the most recent (perhaps 9) entries as well as a 
# blank row to enter new data
# an excel table format might be quite nice.


def query_table_data():
    ### return the dataframes of interest
    end_date = datetime.today()
    start_date = end_date - dateutil.relativedelta.relativedelta(years=1)
    query=f'Select * FROM EffluentLogs WHERE Date BETWEEN "{start_date}" AND "{end_date}"'
    EffluentLogs = db.queryTableDF(query)
    EffluentLogs.index = np.arange(1,len(EffluentLogs)+1)
    EffluentLogs['Date'] = pd.to_datetime(EffluentLogs.Date)
    EffluentLogs.sort_values(by=['Date'], inplace=True, ascending=False)
    EffluentLogs['Date'] = EffluentLogs['Date'].dt.strftime('%d/%m/%Y')
    
    query=f'Select * FROM EffluentInputLogs WHERE Date BETWEEN "{start_date}" AND "{end_date}"'
    EffluentInputLogs = db.queryTableDF(query) 
    EffluentInputLogs.index = np.arange(1,len(EffluentInputLogs)+1)
    EffluentInputLogs['Date'] = pd.to_datetime(EffluentInputLogs.Date)
    EffluentInputLogs.sort_values(by=['Date'], inplace=True, ascending=False)
    EffluentInputLogs['Date'] = EffluentInputLogs['Date'].dt.strftime('%d/%m/%Y')
    
    return EffluentLogs, EffluentInputLogs

def display_table_data(EffluentLog, EffluentInputLogs):
    ### write the data into a ipysheet sheet and format
    sheet1 = from_dataframe(EffluentLog.head(8))
    sheet2 = from_dataframe(EffluentInputLogs.head(8))
    for k,c in enumerate(sheet1.cells):
        c.style['textAlign']='center'
        c.send_state()
    
    return sheet1, sheet2

def update_sheets():
    ### function to update the output of the sheets
    clear_output()
    
    EffluentLogs, EffluentInputLogs = query_table_data()
    sheet1, sheet2 = display_table_data(EffluentLogs, EffluentInputLogs)
    sheet1, out1 = (sheet1, widgets.Output())
    sheet2, out2 = (sheet2, widgets.Output())
    
    with out1:
        display(sheet1)

    with out2:
        display(sheet2)

    button_refresh = widgets.Button(description='Refresh Data')
    button_refresh.on_click(on_button_click)

    out_sheets = widgets.VBox([out1, out2])
    display(out_sheets, button_refresh)
    
def on_button_click(b):
    update_sheets()


update_sheets()

Button(description='Refresh Data', style=ButtonStyle())

# New Effluent Log

In [12]:
def get_irr_vals(start, stop, guns):
    zeros = [int(x) for x in np.zeros(len(paddocks))]
    zeros[3] = 1
    zeros[4] = 1
    zeros[5] = 1
    return zeros

def parse_entry(date, guns, start, stop, speed, storage, notes):
    invalidFlag = False
    # convert fields to appropriate formats
    
    # date, should jsut be a datetime going to text
    dateStr = date.strftime('%Y-%m-%d')
    
    # guns go into first database as string, but also need to become list of numbers
    gunsStr = guns.strip()
    gunsList = guns.split(",")
    gunsList = [int(i) for i in gunsList]
    for g in gunsList:
        if g < 1 or g > 9:
            print("Guns should be entered in the format of Number,Number i.e. 1,2 for guns 1 \
                    and 2. Guns are numbered from 1 to 9.")
            invalidFlag = True
            
    # start and stop should be a string in the paddock list, solved with the combobox widget
    # don't need to do anything for storage and notes
    # add to input log database
    query = f'INSERT INTO EffluentInputLogs (Date, Guns, Start, Stop, Speed, Storage, Notes) \
            VALUES {dateStr, gunsStr, start, stop, speed, storage, notes}'
    
    #print(query)
    db.query(query)
    db.commit()
    
    # parse into Effluent Log database
    # Need tofigure out how to update these meaningfully
    # for now just enter arbitrary data
    paddocks_str = ", ".join( repr(e) for e in paddocks ) 
    vals = get_irr_vals(start, stop, guns)
    valStr = ", ".join(str(int(x)) for x in vals)
    query = f'INSERT INTO EffluentLogs (Date, {paddocks_str}) \
            VALUES ("{dateStr}", {valStr})'
    #print(query)
    db.query(query)
    db.commit()
    
def add_entries_press(b):
    new_entries_arr = to_array(new_log_sheet)
    new_entries_arr[:,0] = [vals.value for vals in new_entries_arr[:,0]]
    new_entries_arr[:,2] = [vals.value for vals in new_entries_arr[:,2]]
    new_entries_arr[:,3] = [vals.value for vals in new_entries_arr[:,3]]
    
    for row in new_entries_arr:
        date, guns, start, stop, speed, storage, notes = row
        if (date is not None and guns is not "" and start is not "" and stop is not ""):
            parse_entry(date, guns, start, stop, speed, storage,  notes)

def setup_sheet():
    new_log_sheet = sheet(rows=5,columns=7, column_headers=['Date', 'Guns', 'Start', 'Stop', 'Speed','Storage',  'Notes'],
                         column_resizing=False, column_width = [5, 3, 3, 3, 3, 3, 9])

    for k,c in enumerate(new_log_sheet.cells):
        c.style['textAlign']='center'
        c.send_state()
        
    column(0, [widgets.DatePicker(layout=widgets.Layout(width='96%')) for _ in range(5)])
    column(1, ["" for _ in range(5)])
    column(2, [widgets.Combobox(options=paddocks, layout=widgets.Layout(width='96%')) for _ in range(5)]) 
    column(3, [widgets.Combobox(options=paddocks, layout=widgets.Layout(width='96%')) for _ in range(5)]) 
    column(4, ["" for _ in range(5)])
    column(5, ["" for _ in range(5)])
    column(6, ["" for _ in range(5)])
    cell(0, 1, "3, 4")
    return new_log_sheet

def clear_sheet(b):
    column(0, [widgets.DatePicker(layout=widgets.Layout(width='96%')) for _ in range(5)])
    column(1, ["" for _ in range(5)])
    column(2, [widgets.Combobox(options=paddocks, layout=widgets.Layout(width='96%')) for _ in range(5)]) 
    column(3, [widgets.Combobox(options=paddocks, layout=widgets.Layout(width='96%')) for _ in range(5)]) 
    column(4, ["" for _ in range(5)])
    column(5, ["" for _ in range(5)])
    column(6, ["" for _ in range(5)])
    

    
new_log_sheet = setup_sheet()
new_log_sheet

button_add = widgets.Button(description='Add Entries')
button_clear = widgets.Button(description='Clear Table')
buttons = widgets.HBox([button_add, button_clear])

button_add.on_click(add_entries_press)
button_clear.on_click(clear_sheet)
widgets.VBox((new_log_sheet, buttons))

In [13]:
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Toggle Code"></form>''')

In [7]:
db.disconnect()

Connection Closed


In [8]:
default_database_location = 'D:\Current Projects\Farm\Effluent Tracker\Effluent Database.db'
db = Database(default_database_location)

Database Connected


# Todo
1. Sort out requriements.txt so that this can run on another device.
3. Figure out cloud hosting for database
4. Input logs
5. Fix graphing so that it rescales if necessary
6. Auto refresh of all widgets?
7. Tes flow of change, commit and rehost.

In [ ]:
import ipywidgets as widgets
from IPython.display import clear_output
outs = widgets.Output()

def mytest(x):
    with outs:
        clear_output()
        lookhere = mytestfilter.value
        if lookhere==1:
            print("hello")
        if lookhere==0:
            print("goodbye")

mytestfilter = widgets.Dropdown(options={'night': 0, 'morning': 1}, description="FILTER")
display(mytestfilter)
display(outs)
mytestfilter.observe(mytest, names='value')

In [26]:
list = [int(x) for x in np.zeros(len(paddocks))]
list[4] = 1
list[5] = 1
list[6] = 1
print(list)


[0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0]


In [19]:
int(3.5)

3